# Example Usage

It's very simple to use this pacakge. You can either import it directly into a python script with `from nbed import nbed`, or run the command `nbed` from the terminal.

## Importing
Here's an example where we've imported the package and pointed it at a `.xyz` file.

In [1]:
from nbed import nbed
from nbed.driver import NbedDriver
from openfermion import count_qubits
from pathlib import Path
import json
from openfermion import get_sparse_operator, get_ground_state

/usr/local/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
#Define all the variables of the molecule
geometry= str(Path('/Users/mac/Desktop/Master/Project/Nbed/tests/molecules/water.xyz').absolute())
n_active_atoms= 1
basis= 'STO-3G'
xc_functional= 'b3lyp'
output= 'openfermion'
projector= 'both'
localization= 'spade'
convergence= 1e-9
transform= 'jordan_wigner'
name = 'water_OS'

with open(geometry, 'r') as infile:
    xyz_string = infile.read()
print(xyz_string)

run_virtual_localization = False
run_fci_emb = True
run_ccsd_emb = True
run_dft_in_dft= False
max_ram_memory = 8_000

max_hf_cycles=5000

charge= 1
spin = 1

occupied_threshold = 0.99
virtual_threshold = 0.95

3

O   0.0000  0.0000  0.0000
H   0.9525  0.0000  0.0000
H   -0.2465  0.9200  0.0000



In [3]:
#Run the NbedDriver class to get the energies and the nbed class to get the qubit hamiltonian
driver = NbedDriver(geometry = xyz_string,
                    n_active_atoms=n_active_atoms,
                    basis = basis,
                    charge = charge,
                    spin = spin,
                    xc_functional = xc_functional,
                    run_virtual_localization = run_virtual_localization,
                    run_fci_emb = run_fci_emb,
                    run_ccsd_emb = run_ccsd_emb,
                    run_dft_in_dft = run_dft_in_dft,
                    max_ram_memory = max_ram_memory,
                    max_hf_cycles=max_hf_cycles,
                    occupied_threshold=occupied_threshold,
                    projector = projector,
                    localization = localization,
                    return_dict = True)

qham = nbed(
    geometry=xyz_string ,
    n_active_atoms=n_active_atoms,
    basis=basis,
    xc_functional=xc_functional,
    output=output,
    projector=projector,
    convergence=convergence,
    localization= localization,
    transform=transform,
    )

1970-01-01 00:00:19,948: nbed.driver: WARNING: Outputting both mu and huzinaga embedding results as tuple.


NbedConfigError: Cannot use 'both' as value of projector.

In [ ]:
#delete pyscf objects and transform numpy arrays to list
#to make it JSON serializable
molecule = driver.molecule_info
for key, val in zip(list(molecule.keys()), list(molecule.values())):
    if str(type(val)) == "<class 'pyscf.scf.hf.RHF'>":
        molecule.pop(key)
    elif str(type(val)) ==  "<class 'pyscf.scf.uhf.UHF'>":
        molecule.pop(key)
    elif str(type(val)) == "<class 'numpy.ndarray'>":
            molecule[key] = val.tolist()

for i in range(len(molecule['mo_energies_emb_post_del'])):
    if str(type(molecule['mo_energies_emb_post_del'][i])) == "<class 'numpy.ndarray'>":
        molecule['mo_energies_emb_post_del'][i] = molecule['mo_energies_emb_post_del'][i].tolist()

In [ ]:
#transform QubitHamiltonian object into a dictionary
ham_dict = {}
n_qubits = count_qubits(qham)
molecule['qubits'] = n_qubits

for term, coeff in qham.terms.items():
    I = list("I"*n_qubits)
    for index, pauli in term:
        I[index] = pauli
    ham_dict[''.join(I)] = coeff
molecule['hamiltonian'] = ham_dict

In [ ]:
#save as a JSON file
with open('/Users/mac/Desktop/Master/Project/Nbed/tests/hamiltonians/' + name + '_' + projector + '.json', 'w') as fp:
    json.dump(molecule, fp)

In [ ]:
#diagonalise the hamiltonian to get the eigenenergies
sparse_op = get_sparse_operator(qham)
get_ground_state(sparse_op)

(-75.11651719312968,
 array([ 1.79339653e-16+4.22602431e-19j, -2.56389411e-17-2.71587509e-17j,
         2.40520658e-19-7.95643017e-18j, ...,
         4.62265295e-16+2.89139256e-16j, -3.79881224e-16+8.98913448e-17j,
         3.28004061e-15+1.52605345e-15j]))